In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


# Module

In [2]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

### Dataset

In [3]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [4]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [5]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [6]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [8]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [9]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [10]:
%%time 

model.fit(x_train_reshaped, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 131us/sample - loss: 0.2075 - accuracy: 0.9345
Epoch 2/10
60000/60000 [==============================] - 5s 87us/sample - loss: 0.0680 - accuracy: 0.9786
Epoch 3/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0503 - accuracy: 0.9841
Epoch 4/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0393 - accuracy: 0.9876
Epoch 5/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0328 - accuracy: 0.9898
Epoch 6/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0282 - accuracy: 0.9907
Epoch 7/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0226 - accuracy: 0.9932
Epoch 8/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0198 - accuracy: 0.9936
Epoch 9/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0167 - accuracy: 0.9944
Epoch 10/10
6

# Model Evaluation

In [11]:
predictions = model.predict(x_test_reshaped)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy*100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on benign test example: 98.97%


# Add noise

In [12]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [13]:
# 1차
noise = np.random.normal(0,0.05,x_test.shape)
noise_fgsm = x_test_norm + noise

for i in range (5):
    new_noise = np.random.normal(0,0.05,x_test.shape)
    x_test_noise = x_test_norm + new_noise
    noise_fgsm = np.concatenate((noise_fgsm,x_test_noise),axis=0)
    
noise_fgsm_reshape = noise_fgsm.reshape(-1,28,28,1)

In [14]:
noise_fgsm_reshape.shape

(60000, 28, 28, 1)

# 1. Projected Gradient Descent (PGD)

# Adversarial Examples

In [15]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=True)

## 1-1) PGD_origin
### 원본 이미지 x_test

In [53]:
%%time 

from art.attacks.evasion import ProjectedGradientDescent
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.1,
                                     batch_size=100)

#Create Adversarial Examples
x_attack_pgd = attack_pgd.generate(x_test_reshaped)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 47 s, sys: 1.47 s, total: 48.5 s
Wall time: 41.4 s


In [54]:
loss,acc = model.evaluate(x_attack_pgd,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 74.04000163078308%


In [55]:
loss,acc = model.evaluate((x_attack_pgd*255),y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 74.77999925613403%


In [56]:
x_pgd_int=(x_attack_pgd*255).astype('int')

In [57]:
loss,acc = model.evaluate(x_pgd_int,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 74.98999834060669%


In [41]:
%%time 

from art.attacks.evasion import ProjectedGradientDescent
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.3,
                                     batch_size=100)

#Create Adversarial Examples
x_attack_pgd = attack_pgd.generate(x_test_reshaped)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 50.9 s, sys: 1.9 s, total: 52.8 s
Wall time: 46.8 s


In [45]:
loss,acc = model.evaluate(x_attack_pgd,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 0.800000037997961%


In [46]:
x_pgd_int=(x_attack_pgd*255).astype('int')

In [47]:
loss,acc = model.evaluate(x_pgd_int,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 0.800000037997961%


# PGD + noise image

In [58]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [61]:
%%time 

from art.attacks.evasion import ProjectedGradientDescent
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.1,
                                     batch_size=100)

#Create Adversarial Examples
x_attack_pgd_noise = attack_pgd.generate(x_test_noise3_reshape)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 49.1 s, sys: 1.68 s, total: 50.7 s
Wall time: 43.5 s


In [62]:
loss,acc = model.evaluate(x_attack_pgd_noise,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 65.2999997138977%


In [63]:
loss,acc = model.evaluate((x_attack_pgd_noise*255),y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 65.7800018787384%


In [64]:
x_pgd_noise_int=(x_attack_pgd_noise*255).astype('int')

In [65]:
loss,acc = model.evaluate(x_pgd_noise_int,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 66.03999733924866%


## 1-1) FGSM_origin
### 원본 이미지 x_test


fgsm 이미지 갯수 늘이기


In [66]:
%%time 

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_reshaped)

CPU times: user 1.45 s, sys: 782 ms, total: 2.23 s
Wall time: 2.51 s


In [67]:
loss,acc = model.evaluate(examples_fgsm,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 81.16999864578247%


In [68]:
loss,acc = model.evaluate((examples_fgsm*255),y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 81.34999871253967%


In [69]:
x_fgsm_noise_int=(examples_fgsm*255).astype('int')

In [70]:
loss,acc = model.evaluate(x_fgsm_noise_int,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 81.54000043869019%


# FGSM + noise image

In [71]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [72]:
%%time 

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm_noise = attack_fgsm.generate(x_test_noise3_reshape)

CPU times: user 1.66 s, sys: 146 ms, total: 1.8 s
Wall time: 1.41 s


In [78]:
loss,acc = model.evaluate(examples_fgsm_noise,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 78.07000279426575%


In [79]:
loss,acc = model.evaluate((examples_fgsm_noise*255),y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 77.63000130653381%


In [76]:
x_fgsm_noise_int=(examples_fgsm_noise*255).astype('int')

In [77]:
loss,acc = model.evaluate(x_fgsm_noise_int,y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 77.74999737739563%
